# **PyTorch 簡介**
PyTorch 是一個機器學習的開發平台，提供使用者實現 MLP, CNN, RNN 等等的深度學習演模型與演算法，以下會介紹 PyTorch 達到深度學習演算法所需的基本概念。

## 本章節內容大綱
* ### [建構支援數值計算的高維度矩陣（Tensor, Multidimensional-array）](#Tensor,Multidimensional-array)
* ### [自動計算微分值（Automatic differentiation）](#AutomaticDifferentiation)
* ### [模型建置以及訓練（Model construction, training）](#ModelConstruction,training)

---

In [2]:
# 匯入套件
import torch
import numpy as np

print(torch.__version__)

2.3.0+cu121


<a name="Tensor,Multidimensional-array"></a>
## 建構支援數值計算的高維度矩陣（Tensor, Multidimensional-array）
PyTorch 張量（torch.Tensor），寫法與 numpy 陣列類似，也能任意從 scalar，list，或 np.ndarray 的型態轉換成 torch.Tensor

In [5]:
a = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32)
print(a)

b = torch.tensor(np.array([1, 2, 3, 4, 5]), dtype=torch.float32)
print(b)

tensor([1., 2., 3., 4., 5.])
tensor([1., 2., 3., 4., 5.])


In [6]:
# torch.Tensor -> np.ndarray
a.numpy()

array([1., 2., 3., 4., 5.], dtype=float32)

* ### Tensor 性質

In [8]:
t = torch.rand(1, 2, 4)

print('數據類型：', t.dtype)   # 數據類型
print('形狀：', t.shape)       # 形狀
print('維度數：', t.ndim)      # 維度數

數據類型： torch.float32
形狀： torch.Size([1, 2, 4])
維度數： 3


* ### Tensor 操作

* #### slice
從 Tensor 中選取部分內容

In [10]:
t = torch.tensor([[0, 1, 2, 3, 4],
                  [5, 6, 7, 8, 9],
                  [10, 11, 12, 13, 14],
                  [15, 16, 17, 18, 19]])

print(t[1:3, 2:])

tensor([[ 7,  8,  9],
        [12, 13, 14]])


In [11]:
t = torch.tensor([[[1, 3, 5, 7],
                  [9, 11, 13, 15]],
                 [[17, 19, 21, 23],
                  [25, 27, 29, 31]]])

print(t[1:2, 1:2, 0:2])

tensor([[[25, 27]]])


* #### reshape
將 Tensor 改變成指定的形狀

In [14]:
t = torch.tensor(
    [[1, 2, 3],
     [4, 5, 6]]
)
print(torch.reshape(t, (6,)))

tensor([1, 2, 3, 4, 5, 6])


In [16]:
t = torch.tensor(
    [[1, 2, 3],
     [4, 5, 6]]
)
print(torch.reshape(t, (3, 2)))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


* #### unsqueeze
增加 Tensor 維度

In [21]:
t = torch.randn((2, 2, 3))

print(torch.unsqueeze(t, 0).shape)

torch.Size([1, 2, 2, 3])


In [22]:
t = torch.randn(2, 2, 3)

print(torch.unsqueeze(t, -1).shape)

torch.Size([2, 2, 3, 1])


* #### squeeze
壓縮 Tensor 維度

In [23]:
t = torch.randn((2, 2, 1))

print(torch.squeeze(t, dim=-1).shape)

torch.Size([2, 2])


In [25]:
t = torch.randn((2, 1, 3, 1))

print(torch.squeeze(t, dim=(1, 3)).shape)

torch.Size([2, 3])


* #### transpose
轉置 Tensor

In [30]:
t = torch.tensor(
    [[1, 2, 3],
     [4, 5, 6]]
)
torch.transpose(t, 0, 1)

tensor([[1, 4],
        [2, 5],
        [3, 6]])

* #### math
數學計算，包括加乘除

In [31]:
a = torch.tensor([[1, 2],
                  [3, 4]])
b = torch.tensor([[1, 1],
                  [1, 1]])

In [32]:
print(a + b, '\n')  # element-wise addition
print(a - b, '\n')  # element-wise subtraction
print(a * b, '\n')  # element-wise multiplication
print(a @ b, '\n')  # matrix multiplication

tensor([[2, 3],
        [4, 5]]) 

tensor([[0, 1],
        [2, 3]]) 

tensor([[1, 2],
        [3, 4]]) 

tensor([[3, 3],
        [7, 7]]) 



In [33]:
print(a + b, "\n")        # element-wise addition
print(a - b, '\n')        # element-wise subtraction
print(a * b, "\n")        # element-wise multiplication
print(torch.matmul(a, b), "\n")  # matrix multiplication

tensor([[2, 3],
        [4, 5]]) 

tensor([[0, 1],
        [2, 3]]) 

tensor([[1, 2],
        [3, 4]]) 

tensor([[3, 3],
        [7, 7]]) 



In [35]:
c = torch.tensor([[4.0, 5.0], [10.0, 1.0]])

# Find the largest value
print(torch.max(c))
# Compute the average value
print(torch.mean(c))
# Find the index of the largest value
print(torch.argmax(c))

tensor(10.)
tensor(5.)
tensor(2)


-------------

<a name="AutomaticDifferentiation"></a>
## 自動計算微分值（Automatic Differentiation）
在深度學習演算法當中，很重要的部分就是如何做模型的更新，其中牽涉到對變數做偏微分。

>若此函數 $f(x) = x^2+3x-5$ 對 $x$ 做偏微分，則能得到 $f^\prime(x) = 2x+3$
>
>將 $x = 1$ 代入函數，得到 $f(x)=-1$，$f^\prime(x) = 5$


In [58]:
def f(x):
    y = x ** 2 + 3 * x - 5
    return y


x = torch.tensor(1.0, requires_grad=True)
print(f(x))

y = f(x)
y.backward()  # 反向傳播計算梯度

g_x = x.grad  # g(x) = f'(x) = dy/dx
print(g_x)

tensor(-1., grad_fn=<SubBackward0>)
tensor(5.)


<a name="ModelConstruction,training"></a>
## 模型建置以及訓練（Model Construction, Training）

* ### 適合新手：

以 toch.nn.Sequential 的方式建構模型以及訓練模型，能夠更快速的建立簡單模型，將會在 Part2，Part3 接續課程中介紹。
* ### 適合專家：

繼承torch.nn.Module 可以更客製化的建置模型、訓練過程等等，於 Part4 做介紹。（在本課程中將會列為選修內容）